# Reading Movies

In [2]:
import ast
import csv
score_scheme = []
with open('ScoreScheme.csv') as f:
    reader = csv.DictReader(f)
    lst = list(reader)
    for item in lst:
        if item["movie_name"] == "":
            continue
        if item["video_url"] == "None":
            continue
        score_scheme.append(item)

In [3]:
len(score_scheme)

2870

In [4]:
movies = []
with open('Output.csv') as f:
    reader = csv.DictReader(f)
    lst = list(reader)
    for item in lst:
        item["query_results"] = ast.literal_eval(item["query_results"])
        movies.append(item)

In [5]:
len(movies)

1967

# Merging Score Scheme File and Movies

In [6]:
movies_with_prerelease = []
for movie in movies:
    if movie["no_prerelease"] == "1":
        continue
    movies_with_prerelease.append(movie)
len(movies_with_prerelease)

1051

In [7]:
index_ignore = []
movies_with_results = []
for movie in movies_with_prerelease:
    movie["results"] = []
    found_movie = False
    for index, result in enumerate(score_scheme):
        if movie["movie_name"].replace(",", "") == result["movie_name"]:
            if index not in index_ignore:
                found_movie = True
                movie["results"].append(result)
                index_ignore.append(index)
        else:
            if found_movie:
                break
    movies_with_results.append(movie)

movies = movies_with_results

In [8]:
pending_to_select_trailers = []
new_movies = []
for movie in movies:
    if movie["results"] == []:
        pending_to_select_trailers.append(movie)
    else:
        new_movies.append(movie)
movies = new_movies
print("Pending to select ",len(pending_to_select_trailers))
print("New movies ",len(new_movies))

Pending to select  39
New movies  1012


In [9]:
movies_with_selected_trailers = []
for movie in movies:
    results = []
    for result in movie["results"]:
        if result["official"] == "" and result["teaser"] == "":
            continue
        results.append(result)
    movie["results"] = results
    movies_with_selected_trailers.append(movie)
movies = movies_with_selected_trailers

In [10]:
new_movies = []
for movie in movies:
    if movie["results"] == []:
        pending_to_select_trailers.append(movie)
    else:
        new_movies.append(movie)
movies = new_movies
print("Pending to select ",len(pending_to_select_trailers))
print("New movies ",len(new_movies))

Pending to select  106
New movies  945


In [11]:
ampliated_movies = []
for movie in movies:
    ampliated_results = []
    for result in movie["results"]:
        not_ampliated_video_id = result["video_url"].replace("https://www.youtube.com/watch?v=", "")
        found = False
        for query_result in movie["query_results"]:
            title, channel, video_id, result_date_str, premiered, stream, subscribers, views, likes = query_result
            if likes == "1like":
                likes = '1'
            if likes == "No" or likes == None:
                likes = '0'
            if not_ampliated_video_id == video_id:
                result["release_date"] = result_date_str
                result["views"] = views
                result["likes"] = likes
                ampliated_results.append(result)
                found = True
                break
        if not found:
            result["release_date"] = "TO-COMPLETE"
            result["views"] = "TO-COMPLETE"
            result["likes"] = "TO-COMPLETE"
            ampliated_results.append(result)
    movie["results"] = ampliated_results
    ampliated_movies.append(movie)
movies = ampliated_movies

In [12]:
completed_movies = []
for movie in movies:
    results = []
    for result in movie["results"]:
        if result["video_url"] == "https://www.youtube.com/watch?v=W6xUnDKcHNA":
            result["release_date"] = '7/23/2013'
            result["views"] = '191,922'
            result["likes"] = '470'
        if result["video_url"] == "https://www.youtube.com/watch?v=0sH0__SpV88":
            result["release_date"] = '3/9/2015'
            result["views"] = '6,768,826'
            result["likes"] = '15,357'
        if result["video_url"] == "https://www.youtube.com/watch?v=HydiAICZq6I":
            result["release_date"] = "6/3/2014"
            result["views"] = "547,288"
            result["likes"] = "971"
        results.append(result)
    movie["results"] = results
    completed_movies.append(movie)
movies = completed_movies

In [13]:
from datetime import date

def days_diff(date1, date2):
    try:
        movie_moth, movie_day, movie_year = date2.split("/")
        result_moth, result_day, result_year = date1.split("/")
        movie_date = date(int(movie_year), int(movie_moth), int(movie_day))
        result_date = date(int(result_year), int(result_moth), int(result_day))
        days_diff_video_movie = (result_date - movie_date).days
    except Exception as e:
        days_diff_video_movie = 1
    return days_diff_video_movie

new_movies = []
for movie in movies:
    results = []
    for result in movie["results"]:
        result["days_diff"] = days_diff(result["release_date"], movie["release_start"])
        results.append(result)
    movie["results"] = results
    new_movies.append(movie)
movies = new_movies

In [14]:
new_movies = []
for movie in movies:
    max_likes = -1
    max_likes_id = None
    for result in movie["results"]:
        if int(result["likes"].replace(",","")) > max_likes:
            max_likes = int(result["likes"].replace(",",""))
            max_likes_id = result["video_url"]

    if max_likes_id == None:
        print("Error")
    results = []
    for result in movie["results"]:
        result["most_liked"] = '0'
        if result["video_url"] == max_likes_id:
            result["most_liked"] = '1'
        results.append(result)
    movie["results"] = results
    new_movies.append(movie)
movies = new_movies

In [15]:
new_movies = []
for movie in movies:
    max_views = -1
    max_views_id = None
    for result in movie["results"]:
        if int(result["views"].replace(",","")) > max_views:
            max_views = int(result["views"].replace(",",""))
            max_views_id = result["video_url"]

    if max_views_id == None:
        print("Error")
    results = []
    for result in movie["results"]:
        result["most_viewed"] = '0'
        if result["video_url"] == max_views_id:
            result["most_viewed"] = '1'
        results.append(result)
    movie["results"] = results
    new_movies.append(movie)
movies = new_movies

In [16]:
new_movies = []
for movie in movies:
    min_days_diff = 901823901823012312
    min_days_diff_id = None
    for result in movie["results"]:
        if int(result["days_diff"]) < min_days_diff:
            min_days_diff_id = int(result["days_diff"])
            min_days_diff_id = result["video_url"]

    if min_days_diff_id == None:
        print("Error")

    results = []
    for result in movie["results"]:
        result["oldest"] = '0'
        if result["video_url"] == min_days_diff_id:
            result["oldest"] = '1'
        results.append(result)
    movie["results"] = results
    new_movies.append(movie)
movies = new_movies

In [17]:
new_movies = []
for movie in movies:
    max_days_diff = -12389018230912839012381029
    max_days_diff_id = None
    for result in movie["results"]:
        if int(result["days_diff"]) > max_days_diff:
            max_days_diff = int(result["days_diff"])
            max_days_diff_id = result["video_url"]

    if max_days_diff_id == None:
        print("Error")

    results = []
    for result in movie["results"]:
        result["most_recent"] = '0'
        if result["video_url"] == max_days_diff_id:
            result["most_recent"] = '1'
        results.append(result)
    movie["results"] = results
    new_movies.append(movie)
movies = new_movies

In [23]:
with open("./selected_trailers.csv", "w") as f:
    f.write(
        "uniqueID,movie_name,num_trailers,trailer_id,video_url,video_title,channel,official,teaser,days_diff,trailer_release,trailer_view,trailer_like,trailer_comment,most_recent,oldest,most_liked,most_viewed,most_comment")
    f.write("\r\n")
    for movie in movies:
        movie_name = movie["movie_name"].replace(",", "")
        id = 0
        for result in movie["results"]:
            official = 0
            teaser = 0
            if result["teaser"] == "1":
                teaser = 1
            if result["official"] == "1":
                official = 1
            f.write("{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}".format(
                movie["uniqueID"],
                movie_name,
                len(movie["results"]),
                id,
                result["video_url"],
                result["video_title"],
                result["channel"],
                official,
                teaser,
                result["days_diff"],
                result["release_date"],
                result["views"].replace(",", ""),
                result["likes"].replace(",", ""),
                "TO-DO",
                result["most_recent"],
                result["oldest"],
                result["most_liked"],
                result["most_viewed"],
                "TO-DO",
            ))
            f.write("\r\n")
            id = id + 1

In [19]:
for movie in pending_to_select_trailers:
    #if movie["no_prerelease"] == "1":
    #    print("Error")
    print(movie["movie_name"])

The Dark Knight
Fast & Furious
Clash of the Titans
Watchmen
Superman Returns
The Divergent Series: Insurgent
Ghost Rider
Happy Feet
Casino Royale
Cloverfield
Chicken Little
Get Smart
Inglourious Basterds
Robin Hood
10,000 BC
I Now Pronounce You Chuck & Larry
Live Free or Die Hard
A Nightmare on Elm Street
Four Christmases
Knocked Up
The Curious Case of Benjamin Button
TMNT
Unstoppable
Bridge to Terabithia
The Forbidden Kingdom
Bride Wars
1408
Deja Vu
Pinocchio
The Letters
The Abandoned
Blindness
Persecuted
The Last Legion
Charlie Bartlett
Burnt
Hot Rod
Flash of Genius
The Notebook II
Spider-Man 3
Iron Man 2
Shrek the Third
Pirates of the Caribbean: At World's End
Transformers: Revenge of the Fallen
Indiana Jones and the Kingdom of the Crystal Skull
Transformers: Dark of the Moon
Pirates of the Caribbean: On Stranger Tides
I Am Legend
Harry Potter and the Order of the Phoenix
Star Trek
The Simpsons Movie
300
Shrek Forever After
Twilight
Maleficent
The Bourne Ultimatum
Up
World War Z
Tee